# Capstone II: Craiglist Used Car Dataset
### Part 3: Preprocessing and Model Development
Biman Mondal

May 16, 2025

Original Dataset .csv file downloaded from Kaggle:
https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data

The goal of this capstone project is to create a model that predicts the price of a vehicle given certain dependent variables.

Step 1: Data Wrangling
Wrangled data cleaned in: CapstoneTwo_CLCarDataset_DataWrangling.ipynb
The original dataset has been *halved* to speed up reading and processing.

Step 2: EDA
The nan values have been removed where possible and imputed other places. In the mileage (odometer) column, the year and condition were used to provide the replace the null/nan values with the median.
The columns not likely to be useful in predicting price were dropped from the dataframe.

Step 3: Preprocessing 
In this notebook, further steps are taken to scrub the dataset of N/A values, imputing common values, and dropping outliers
The categorical features of the dataset will be encoded in preparation for modeling

In [ ]:
#!pip install category_encoders

In [ ]:
# Import numpy package
import numpy as np
# Import pandas package
import pandas as pd
# Import matplotlib package
import matplotlib.pyplot as plt
# Import the seaborn package
import seaborn as sns
# Import ydata-profiling package
# Import datetime
import datetime as dt
# Import category encoder
import category_encoders as ce

In [ ]:
vehicles = pd.read_csv('./data/vehicles_eda.csv', header='infer',index_col='id')
#vehicles.set_index('id', inplace=True)

In [ ]:
#vehicles = vehicles.drop([['Unnamed: 0','url', axis=1)
vehicles.sample(5)

In [ ]:
# Drop all the unnecessary columns
#vehicles.drop(model_na.index,inplace=True)
#print(help(vehicles.drop))
# Columns to drop
drop_cols = ['region','county','state','lat','long','posting_date','fuel_code','condition_code']
vehicles = vehicles.drop(drop_cols,axis=1)

In [ ]:
vehicles.info()

### Remove all NA values

In [ ]:
### Model
#dir(vehicles['manufacturer'].count())
foo = vehicles[['manufacturer','model']]

In [ ]:
model_na = vehicles.loc[vehicles['model'].isna(),['model']]
print(f'\nRemove {len(model_na)} rows with no model name.\n')
#print(model_na.index)
vehicles.drop(model_na.index,inplace=True)

In [ ]:
### Manufacturer
manufacturer_na = vehicles.loc[vehicles['manufacturer'].isna(),['manufacturer','model']]
print(f'\nRemove {len(manufacturer_na)} rows with no model name.\n')
#manufacturer_na
vehicles.drop(manufacturer_na.index,inplace=True)

In [ ]:
vehicles.info()

In [ ]:
### Drive
#print(help(vehicles['drive'].hist(align='mid')))
#vehicles['drive'].hist(align='left',edgecolor='black')
#plt.show()
drive_na = vehicles.loc[vehicles['drive'].isna(),['manufacturer','model','type','drive']]
print(f'\nThere are {len(drive_na)} rows without the drive type specified.\n')
drive_na.sample(10)
vehicles['drive'] = vehicles['drive'].fillna('fwd')

In [ ]:
### Transmission
#vehicles['transmission'].hist(rwidth=1.0,align='left')
#plt.show()
transmission_na = vehicles.loc[vehicles['transmission'].isna(),['manufacturer','model','type','transmission']]
transmission_na.sample(10)
vehicles['transmission'] = vehicles['transmission'].fillna('automatic')

In [ ]:
### Type
#vehicles.loc[vehicles['type']=='offroad',['year','manufacturer','model','type']]
type_map = {'offroad':'SUV','hatchback':'sedan','coupe':'sedan','convertible':'sedan','wagon':'sedan'}
vehicles['type'] = vehicles['type'].replace(type_map)

In [ ]:
#ax = vehicles['type'].hist(align='mid',rwidth=0.5)
#ax.set_xticklabels(ax.get_xticklabels(),rotation=60)
#plt.show()
vehicles['type'] = vehicles['type'].fillna('sedan')
vehicles.info()

In [ ]:
### Title Status
vehicles['title_status'].unique()

na_title_status = vehicles.loc[vehicles['title_status'].isna(),['manufacturer','model','title_status']]
na_title_status
# Replace na values in title_status
vehicles['title_status'] = vehicles['title_status'].fillna('clean')

# Replace missing and lien to clean
vehicles['title_status'] = vehicles['title_status'].replace({'missing': 'clean'})

In [ ]:
#vehicles['title_status'].hist(log=True)
#plt.show()

### Condense the model category 
Since there are so many model categories, they should be condensed to simplify categorization.

Assign standard labels for the top 40 vehicle models including for f-150, f-250, silverado, and other.

In [ ]:
print(f"There are {vehicles['model'].nunique()} unique models in the dataset!!")
print(vehicles['model'].value_counts().nlargest(20))
foo=vehicles['model'].value_counts()

In [ ]:
vehicles.loc[(vehicles.model.str.contains("f150"))&(vehicles.manufacturer=='ford'),'model'] = 'f-150'
f150_pattern ="f-[150]."
#vehicles.loc[vehicles.model.str.contains(f150_pattern),'model'] = 'f-150'
#vehicles.loc[((vehicles.model.str.contains(f150_pattern))&(vehicles.manufacturer=='ford')),['manufacturer','model']]
vehicles.loc[((vehicles.model.str.contains(f150_pattern))&(vehicles.manufacturer=='ford')),'model']='f-150'

In [ ]:
# Combine the f-250 model segment
vehicles.loc[vehicles.model.str.contains('f.250.'), 'model']
vehicles.loc[(vehicles.model.str.contains('.f*.250.')) & (vehicles.manufacturer=='ford'),'model'] = 'f-250'
vehicles.loc[vehicles.model.str.contains('f*.250.') & (vehicles.manufacturer=='ford'),'model'] = 'f-250'
vehicles.loc[vehicles.model.str.contains('f250') & (vehicles.manufacturer=='ford'),'model'] = 'f-250'

In [ ]:
# Combine all f-350 model segments
vehicles.loc[(vehicles.model.str.contains('.f-350.')) & (vehicles.manufacturer=='ford'),'model'] = 'f-350'
vehicles.loc[(vehicles.model.str.contains('.350.')) & (vehicles.manufacturer=='ford'),'model'] = 'f-350'
vehicles.loc[(vehicles.model.str.contains('f350')) & (vehicles.manufacturer=='ford'),'model'] = 'f-350'

In [ ]:
# Combine all the silverado 1500 
vehicles.loc[(vehicles.model.str.contains('.*silverado 1500.*')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 1500'
vehicles.loc[(vehicles.model.str.contains('1500')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 1500'
vehicles.loc[(vehicles.model.str.contains('silverado$')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 1500'
vehicles.loc[(vehicles.model.str.contains('.*1500.*')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 1500'
#vehicles.loc[vehicles.model.str.contains('silverado'), ['model','manufacturer']]

In [ ]:
# Combine all the silverado 2500 
vehicles.loc[(vehicles.model.str.contains('.*silverado 2500.*')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 2500'
vehicles.loc[(vehicles.model.str.contains('2500')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 2500'
vehicles.loc[(vehicles.model.str.contains('.*silverado 3500.*')) & (vehicles.manufacturer=='chevrolet'),'model'] = 'silverado 3500'

In [ ]:
# Combine all ram 1500 
vehicles.loc[(vehicles.model.str.contains('.*1500.*')) & (~vehicles.model.str.contains('.*van.*')) & (vehicles.manufacturer=='ram'),'model'] = '1500'
vehicles.loc[(vehicles.model.str.contains('.*2500.*')) & (~vehicles.model.str.contains('.*van.*'))& (vehicles.manufacturer=='ram'),'model'] = '2500'
vehicles.loc[(vehicles.model.str.contains('.*3500.*')) & (~vehicles.model.str.contains('.*van.*'))& (vehicles.manufacturer=='ram'),'model']  = '3500'
vehicles.loc[vehicles.model.str.contains('3500'), ['model','manufacturer']]

In [ ]:
vehicles.loc[(vehicles.model.str.contains('.*wrangler.*')) & (vehicles.manufacturer=='jeep'),'model'] = 'wrangler'
#vehicles.loc[vehicles.model.str.contains('.*wrangler.*'), ['model','manufacturer']]

In [ ]:
# Combine toyota tacoma models
vehicles.loc[vehicles.model.str.contains('tacoma'), 'model'] = 'tacoma'
# Combine toyota tundra models
vehicles.loc[vehicles.model.str.contains('tundra'), 'model'] = 'tundra'
vehicles.loc[vehicles.model.str.contains('.*camry.*'), 'model'] = 'camry'
vehicles.loc[vehicles.model.str.contains('.*corolla.*'), 'model'] = 'corolla'
#vehicles.loc[vehicles.model.str.contains('tundra'), ['model','manufacturer']]

In [ ]:
# Combine Honda Civic and Accord
vehicles.loc[vehicles.model.str.contains('.*accord.*'), 'model'] = 'accord'
vehicles.loc[vehicles.model.str.contains('.*civic.*'), 'model'] = 'civic'
vehicles.loc[vehicles.model.str.contains('.*escape.*'), 'model'] = 'escape'
vehicles.loc[vehicles.model.str.contains('.*altima.*'), 'model'] = 'altima'
vehicles.loc[vehicles.model.str.contains('.*explorer.*'), 'model'] = 'explorer'
vehicles.loc[vehicles.model.str.contains('.*cherokee.*'), 'model'] = 'cherokee'
vehicles.loc[vehicles.model.str.contains('.*mustang.*'), 'model'] = 'mustang'
vehicles.loc[vehicles.model.str.contains('.*fusion.*'), 'model'] = 'fusion'
vehicles.loc[vehicles.model.str.contains('.*cr-v.*'), 'model'] = 'cr-v'
vehicles.loc[vehicles.model.str.contains('.*focus.*'), 'model'] = 'focus'
vehicles.loc[vehicles.model.str.contains('.*outback.*'), 'model'] = 'outback'
vehicles.loc[vehicles.model.str.contains('.*impala.*'), 'model'] = 'impala'
vehicles.loc[vehicles.model.str.contains('.*malibu.*'), 'model'] = 'malibu'
vehicles.loc[vehicles.model.str.contains('.*rav4.*'), 'model'] = 'rav4'
vehicles.loc[vehicles.model.str.contains('.*corvette.*'), 'model'] = 'corvette'
vehicles.loc[vehicles.model.str.contains('.*charger.*'), 'model'] = 'charger'
vehicles.loc[vehicles.model.str.contains('.*grand caravan.*'), 'model'] = 'grand caravan' 
vehicles.loc[vehicles.model.str.contains('.*sierra 1500.*'), 'model'] = 'sierra 1500'
vehicles.loc[vehicles.model.str.contains('.*tahoe.*'), 'model'] = 'tahoe'
vehicles.loc[vehicles.model.str.contains('.*odyssey.*'), 'model'] = 'odyssey'
vehicles.loc[vehicles.model.str.contains('.*cruze.*'), 'model'] = 'cruze'
vehicles.loc[vehicles.model.str.contains('.*sonata.*'), 'model'] = 'sonata'
vehicles.loc[vehicles.model.str.contains('.*jetta.*'), 'model'] = 'jetta'
vehicles.loc[vehicles.model.str.contains('.*prius.*'), 'model'] = 'prius'
vehicles.loc[vehicles.model.str.contains('.*edge.*'), 'model'] = 'edge'
vehicles.loc[vehicles.model.str.contains('.*elantra.*'), 'model'] = 'elantra'

In [ ]:
# The following are the top 40 car models 
print(vehicles['model'].value_counts().nlargest(40))

In [ ]:
top_models = vehicles['model'].value_counts().nlargest(40)
top_models.index
vehicles_sub = vehicles[vehicles['model'].isin(top_models.index)]
print(vehicles.shape)
print(vehicles_sub.shape)

### Numerical Value distribution

In [ ]:
# Make subplots
#fig, axes = plt.subplots(2,2, figsize=(5, 5))
print(vehicles['price'].quantile(0.99))

vehicles = vehicles[vehicles['price']<100000]

#print(vehicles['price'].describe())
plt.hist(vehicles['price'],bins=20)
plt.show()

In [ ]:
print(vehicles['year'].quantile(0.05))
vehicles = vehicles[vehicles['year']>1990]
plt.hist(vehicles['year'],bins=30)
plt.show()

In [ ]:
print(vehicles['odometer'].quantile(0.95))
vehicles = vehicles[vehicles['odometer']<300000]
plt.hist(vehicles['odometer'],bins=20)
plt.show()

In [ ]:
#sns.histplot(vehicles['cylinders'],bins=6)
#plt.show()

In [ ]:
#sns.pairplot(vehicles[['price','year','cylinders','odometer']])
#plt.show()

### OHE The Data


In [ ]:
# Encode data
# Scale data to understand distribution?

In [ ]:
vehicles['condition'].unique()

In [ ]:
# Condition is categorical so it is suitable for single column mapping
cond_map = {'new': 1, 'like new': 2, 'excellent': 3, 'good': 4, 'fair': 5}
vehicles['condition_num'] = vehicles['condition'].map(cond_map)

In [ ]:
# Map The manufacturer

In [ ]:
print(vehicles_sub['manufacturer'].nunique())
print(vehicles_sub['manufacturer'].unique())

encoder = ce.OneHotEncoder(cols=['manufacturer','model','fuel','title_status','transmission','drive','type'], 
                           use_cat_names=True,
                            return_df=True)
vehicle_transformed = encoder.fit_transform(vehicles_sub)
vehicle_transformed.head()
print(vehicle_transformed.shape)

### Run sample regression analysis

In [ ]:
#vehicle_transformed = vehicle_transformed.drop(['condition','manufacturer','model','fuel','title_status','transmission','drive','type'],axis=1)
#vehicle_transformed.head()
#vehicle_transformed=vehicle_transformed.drop('condition',axis=1)

In [ ]:
# Import regression modules
import statsmodels.api as sm

In [340]:
X = vehicle_transformed.drop('price',axis=1)
X = sm.add_constant(X)
m = sm.OLS(vehicle_transformed['price'],X).fit()
print(m.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.744
Method:                 Least Squares   F-statistic:                     2610.
Date:                Tue, 20 May 2025   Prob (F-statistic):               0.00
Time:                        15:51:26   Log-Likelihood:            -5.9585e+05
No. Observations:               57526   AIC:                         1.192e+06
Df Residuals:                   57461   BIC:                         1.192e+06
Df Model:                          64                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     

### Summary

In this notebook, the vehicle dataset was fully cleaned of N/A values. The numerical features were trimmed of the extreme values. The dataset has been prepared for modeling by encoding the remaining categorical features. 

The "model" feature of the dataset poses a significant challenges to modeling as there are over 10,000 unique categories. The "model" feature was filtered for the top 40 common models and standard labels were applied manually. The rows not associated with the top 40 models were then dropped. This results in a much smaller dataset. A more optimum manner to condense the "model" feature will need to be investigated.

The reduced dataset was OHE using category_encoders library. The preliminary linear regression fit shows a cleaner fit than during the EDA step, suggesting the dataset is properly prepared for modeling.  

Note that the original vehicle dataset was halved for speedy run times. For the next step, the original full dataset will be used to increase number of datapoints used for modeling. 